**PREDICTIVE TERMINATION CRITERION**

This notebook aims to experiment the predictive termination criterion described in "Domhan, Tobias, Jost Tobias Springenberg, and Frank Hutter. "Speeding up automatic hyperparameter optimization of deep neural networks by extrapolation of learning curves." Twenty-fourth international joint conference on artificial intelligence. 2015." on several Skitlearn learners and datasets from OpenML


**Fullfill requirements**

In [3]:
!pip install emcee
!pip install triangle
!pip install openml

     |████████████████████████████████| 45 kB 1.7 MB/s 
     |████████████████████████████████| 1.9 MB 5.1 MB/s 
     |████████████████████████████████| 119 kB 5.0 MB/s 
     |████████████████████████████████| 75 kB 3.5 MB/s 
  Created wheel for openml: filename=openml-0.12.2-py3-none-any.whl size=137327 sha256=7743aba6070807b1a801c454e743142c23eef8cd4f38d1c4d008e2dc9d92849f
  Stored in directory: /root/.cache/pip/wheels/6a/20/88/cf4ac86aa18e2cd647ed16ebe274a5dacee9d0075fa02af250
  Created wheel for liac-arff: filename=liac_arff-2.5.0-py3-none-any.whl size=11731 sha256=2e9a6451f1350687ed90abb980fe6742e5c6891bfebf3e8c0dee8149c9f4bdd7
  Stored in directory: /root/.cache/pip/wheels/1f/0f/15/332ca86cbebf25ddf98518caaf887945fbe1712b97a0f2493b
Successfully built openml liac-arff


**Import section**

In [13]:
import time
import math
import emcee
import openml
import random
import logging
import inspect
import triangle
import traceback
import numpy as np
from pylab import *
from functools import reduce
from scipy.stats import norm
from functools import partial
from sklearn.base import clone
from scipy.stats import norm, kde
from sklearn.svm import LinearSVC
from scipy.special import logsumexp
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from scipy.optimize import curve_fit, leastsq, fmin_bfgs, fmin_l_bfgs_b, nnls
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, PassiveAggressiveClassifier, Perceptron, RidgeClassifier, SGDClassifier

**Curve Functions**

In [5]:
#all the models that we considered at some point
all_models = {}
model_defaults = {}
display_name_mapping = {}


def pow3(x, c, a, alpha):
    return c - a * x**(-alpha)
all_models["pow3"] = pow3
model_defaults["pow3"] = {"c": 0.84, "a": 0.52, "alpha": 0.01}
display_name_mapping["pow3"] = "pow$_3$"

def linear(x, a, b):
    return a*x + b
#models["linear"] = linear
all_models["linear"] = linear

"""
    Source: curve expert
"""
def log_power(x, a, b, c):
    #logistic power
    return a/(1.+(x/np.exp(b))**c)
all_models["log_power"] = log_power
model_defaults["log_power"] = {"a": 0.77, "c": -0.51, "b": 2.98}
display_name_mapping["log_power"] = "log power"

def weibull(x, alpha, beta, kappa, delta):
    """
    Weibull modell

    http://www.pisces-conservation.com/growthhelp/index.html?morgan_mercer_floden.htm

    alpha: upper asymptote
    beta: lower asymptote
    k: growth rate
    delta: controls the x-ordinate for the point of inflection
    """
    return alpha - (alpha - beta) * np.exp(-(kappa * x)**delta)
all_models["weibull"] = weibull
model_defaults["weibull"] = {"alpha": .7, "beta": 0.1, "kappa": 0.01,  "delta": 1}
display_name_mapping["weibull"] = "Weibull"


def mmf(x, alpha, beta, kappa, delta):
    """
        Morgan-Mercer-Flodin

        description:
        Nonlinear Regression page 342
        http://bit.ly/1jodG17
        http://www.pisces-conservation.com/growthhelp/index.html?morgan_mercer_floden.htm  

        alpha: upper asymptote
        kappa: growth rate
        beta: initial value
        delta: controls the point of inflection
    """
    return alpha - (alpha - beta) / (1. + (kappa * x)**delta)
all_models["mmf"] = mmf
model_defaults["mmf"] = {"alpha": .7, "kappa": 0.01, "beta": 0.1, "delta": 5}
display_name_mapping["mmf"] = "MMF"

def janoschek(x, a, beta, k, delta):
    """
        http://www.pisces-conservation.com/growthhelp/janoschek.htm
    """
    return a - (a - beta) * np.exp(-k*x**delta)
all_models["janoschek"] = janoschek
model_defaults["janoschek"] = {"a": 0.73, "beta": 0.07, "k": 0.355, "delta": 0.46}
display_name_mapping["janoschek"] = "Janoschek"

def ilog2(x, c, a):
    x = 1 + x
    assert(np.all(x>1))
    return c - a / np.log(x)
all_models["ilog2"] = ilog2
model_defaults["ilog2"] = {"a": 0.43, "c": 0.78}
display_name_mapping["ilog2"] = "ilog$_2$"


def dr_hill_zero_background(x, theta, eta, kappa):
    return (theta* x**eta) / (kappa**eta + x**eta)
all_models["dr_hill_zero_background"] = dr_hill_zero_background
model_defaults["dr_hill_zero_background"] = {"theta": 0.772320, "eta": 0.586449, "kappa": 2.460843}
display_name_mapping["dr_hill_zero_background"] = "Hill$_3$"


def logx_linear(x, a, b):
    x = np.log(x)
    return a*x + b
all_models["logx_linear"] = logx_linear
model_defaults["logx_linear"] = {"a": 0.378106, "b": 0.046506}
display_name_mapping["logx_linear"] = "log x linear"


def vap(x, a, b, c):
    """ Vapor pressure model """
    return np.exp(a+b/x+c*np.log(x))
all_models["vap"] = vap
model_defaults["vap"] = {"a": -0.622028, "c": 0.042322, "b": -0.470050}
display_name_mapping["vap"] = "vapor pressure"



def loglog_linear(x, a, b):
    x = np.log(x)
    return np.log(a*x + b)
all_models["loglog_linear"] = loglog_linear
display_name_mapping["loglog_linear"] = "log log linear"


#Models that we chose not to use in the ensembles/model combinations:

#source: http://aclweb.org/anthology//P/P12/P12-1003.pdf
def exp3(x, c, a, b):
    return c - np.exp(-a*x+b)
all_models["exp3"] = exp3
model_defaults["exp3"] = {"c": 0.7, "a": 0.01, "b": -1}
display_name_mapping["exp3"] = "exp$_3$"


def exp4(x, c, a, b, alpha):
    return c - np.exp(-a*(x**alpha)+b)
all_models["exp4"] = exp4
model_defaults["exp4"] = {"c": 0.7, "a":  0.8, "b":-0.8, "alpha": 0.3}
display_name_mapping["exp4"] = "exp$_4$"

def pow2(x, a, alpha):
    return a * x**(-alpha)
all_models["pow2"] = pow2
model_defaults["pow2"] = {"a": 0.1, "alpha": -0.3}
display_name_mapping["pow2"] = "pow$_2$"

def pow4(x, c, a, b, alpha):
    return c - (a*x+b)**-alpha
all_models["pow4"] = pow4
model_defaults["pow4"] = {"alpha": 0.1, "a": 200, "b": 0., "c": 0.8}
display_name_mapping["pow4"] = "pow$_4$"



def sat_growth(x, a, b):
    return a * x / (b + x)
all_models["sat_growth"] = sat_growth
model_defaults["sat_growth"] = {"a": 0.7, "b": 20}
display_name_mapping["sat_growth"] = "saturated growth rate"


def dr_hill(x, alpha, theta, eta, kappa):
    return alpha + (theta*(x**eta)) / (kappa**eta + x**eta)
all_models["dr_hill"] = dr_hill
model_defaults["dr_hill"] = {"alpha": 0.1, "theta": 0.772320, "eta": 0.586449, "kappa": 2.460843}
display_name_mapping["dr_hill"] = "Hill$_4$"



def gompertz(x, a, b, c):
    """
        Gompertz growth function.
        
        sigmoidal family
        a is the upper asymptote, since
        b, c are negative numbers
        b sets the displacement along the x axis (translates the graph to the left or right)
        c sets the growth rate (y scaling)
        
        e.g. used to model the growth of tumors
        
        http://en.wikipedia.org/wiki/Gompertz_function
    """
    return a*np.exp(-b*np.exp(-c*x))
    #return a + b * np.exp(np.exp(-k*(x-i)))
all_models["gompertz"] = gompertz
model_defaults["gompertz"] = {"a": 0.8, "b": 1000, "c": 0.05}
display_name_mapping["gompertz"] = "Gompertz"


def logistic_curve(x, a, k, b):
    """
        a: asymptote
        k: 
        b: inflection point
        http://www.pisces-conservation.com/growthhelp/logistic_curve.htm
    """
    return a / (1. + np.exp(-k*(x-b)))
all_models["logistic_curve"] = logistic_curve
model_defaults["logistic_curve"] = {"a": 0.8, "k": 0.01, "b": 1.}
display_name_mapping["logistic_curve"] = "logistic curve"



def bertalanffy(x, a, k):
    """
        a: asymptote
        k: growth rate
        http://www.pisces-conservation.com/growthhelp/von_bertalanffy.htm
    """
    return a * (1. - np.exp(-k*x))
all_models["bertalanffy"] = bertalanffy
model_defaults["bertalanffy"] = {"a": 0.8, "k": 0.01}
display_name_mapping["bertalanffy"] = "Bertalanffy"



curve_combination_models_old = ["vap", "ilog2", "weibull", "pow3", "pow4", "loglog_linear",
                                "mmf", "janoschek", "dr_hill_zero_background", "log_power",
                                "exp4"]


curve_combination_models = ["weibull", "pow4", "mmf", "pow3", "loglog_linear",
                            "janoschek", "dr_hill_zero_background", "log_power", "exp4"]

curve_ensemble_models = ["vap", "ilog2", "weibull", "pow3", "pow4", "loglog_linear",
                         "mmf", "janoschek", "dr_hill_zero_background", "log_power",
                         "exp4"]

**Curve Models**

In [6]:
def recency_weights(num):
    if num == 1:
        return np.ones(1)
    else:
        recency_weights = [10**(1./num)] * num
        recency_weights = recency_weights**(np.arange(0, num))
        return recency_weights


def masked_mean_x_greater_than(posterior_distribution, y):
    """
        P(E[f(x)] > E[y] | Data)
    """
    predictions = np.ma.masked_invalid(posterior_distribution)
    return np.sum(predictions > y) / float(np.sum(~predictions.mask))


class CurveModel(object):

    def __init__(self,
                 function,
                 function_der=None,
                 min_vals={},
                 max_vals={},
                 default_vals={}):
        """
            function: the function to be fit
            function_der: derivative of that function
        """
        self.function = function
        if function_der != None:
            raise NotImplementedError("function derivate is not implemented yet...sorry!")
        self.function_der = function_der
        assert isinstance(min_vals, dict)
        self.min_vals = min_vals.copy()
        assert isinstance(max_vals, dict)
        self.max_vals = max_vals.copy()
        function_args = inspect.getargspec(function).args
        assert "x" in function_args, "The function needs 'x' as a parameter."
        for default_param_name in default_vals.keys():
            if default_param_name == "sigma":
                continue
            msg = "function %s doesn't take default param %s" % (function.__name__, default_param_name)
            assert default_param_name in function_args, msg
        self.function_params = [param for param in function_args if param != 'x']
        #set default values:
        self.default_vals = default_vals.copy()
        for param_name in self.function_params:
            if param_name not in default_vals:
                print("setting function parameter %s to default of 1.0 for function %s" % (param_name,
                                                                                           self.function.__name__))
                self.default_vals[param_name] = 1.0
        self.all_param_names = [param for param in self.function_params]
        self.all_param_names.append("sigma")
        self.name = self.function.__name__
        self.ndim = len(self.all_param_names)
        
        #uniform noise prior over interval:
        if "sigma" not in self.min_vals:
            self.min_vals["sigma"] = 0.
        if "sigma" not in self.max_vals:
            self.max_vals["sigma"] = 1.0
        if "sigma" not in self.default_vals:
            self.default_vals["sigma"] = 0.05
    
    def default_function_param_array(self):
        return np.asarray([self.default_vals[param_name] for param_name in self.function_params])

    def are_params_in_bounds(self, theta):
        """
            Are the parameters in their respective bounds?
        """
        in_bounds = True
        
        for param_name, param_value in zip(self.all_param_names, theta):
            if param_name in self.min_vals:
                if param_value < self.min_vals[param_name]:
                    in_bounds = False
            if param_name in self.max_vals:
                if param_value > self.max_vals[param_name]:
                    in_bounds = False
        return in_bounds

    def split_theta(self, theta):
        """Split theta into the function parameters (dict) and sigma. """
        params = {}
        sigma = None
        for param_name, param_value in zip(self.all_param_names, theta):
            if param_name in self.function_params:
                params[param_name] = param_value
            elif param_name == "sigma":
                sigma = param_value
        return params, sigma

    def split_theta_to_array(self, theta):
        """Split theta into the function parameters (array) and sigma. """
        params = theta[:-1]
        sigma = theta[-1]
        return params, sigma

    def fit(self, x, y):
        raise NotImplementedError()

    def predict(self, x):
        raise NotImplementedError()

    def predict_given_theta(self, x, theta):
        """
            Make predictions given a single theta
        """
        params, sigma = self.split_theta(theta)
        predictive_mu = self.function(x, **params)
        return predictive_mu, sigma

    def likelihood(self, x, y):
        """
            for each y_i in y:
                p(y_i|x, model)
        """
        params, sigma = self.split_theta(self.ml_params)
        return norm.pdf(y-self.function(x, **params), loc=0, scale=sigma)


class MLCurveModel(CurveModel):
    """
        ML fit of a curve.
    """

    def __init__(self, recency_weighting=True,  **kwargs):
        super(MLCurveModel, self).__init__(**kwargs)

        #Maximum Likelihood values of the parameters
        self.ml_params = None
        self.recency_weighting = recency_weighting

    def fit(self, x, y, weights=None, start_from_default=True):
        """
            weights: None or weight for each sample.
        """
        return self.fit_ml(x, y, weights, start_from_default)

    def predict(self, x):
        #assert len(x.shape) == 1
        params, sigma = self.split_theta_to_array(self.ml_params)
        return self.function(x, *params)
        #return np.asarray([self.function(x_pred, **params) for x_pred in x])

    def fit_ml(self, x, y, weights, start_from_default):
        """
            non-linear least-squares fit of the data.

            First tries Levenberg-Marquardt and falls back
            to BFGS in case that fails.

            Start from default values or from previous ml_params?
        """
        successful = self.fit_leastsq(x, y, weights, start_from_default)
        if not successful:
            successful = self.fit_bfgs(x, y, weights, start_from_default)
            if not successful:
                return False
        return successful

    def ml_sigma(self, x, y, popt, weights):
        """
            Given the ML parameters (popt) get the ML estimate of sigma.
        """
        if weights is None:
            if self.recency_weighting:
                variance = np.average((y-self.function(x, *popt))**2,
                    weights=recency_weights(len(y)))
                sigma = np.sqrt(variance)
            else:
                sigma = (y-self.function(x, *popt)).std()
        else:
            if self.recency_weighting:
                variance = np.average((y-self.function(x, *popt))**2,
                    weights=recency_weights(len(y)) * weights)
                sigma = np.sqrt(variance)
            else:
                variance = np.average((y-self.function(x, *popt))**2,
                    weights=weights)
                sigma = np.sqrt(variance)
        return sigma

    def fit_leastsq(self, x, y, weights, start_from_default):    
        try:
            if weights is None:
                if self.recency_weighting:
                    residuals = lambda p: np.sqrt(recency_weights(len(y))) * (self.function(x, *p) - y)
                else:
                    residuals = lambda p: self.function(x, *p) - y
            else:
                #the return value of this function will be squared, hence
                #we need to take the sqrt of the weights here
                if self.recency_weighting:
                    residuals = lambda p: np.sqrt(recency_weights(len(y))*weights) * (self.function(x, *p) - y)
                else:
                    residuals = lambda p: np.sqrt(weights) * (self.function(x, *p) - y)

            
            if start_from_default:
                initial_params = self.default_function_param_array()
            else:
                initial_params, _ = self.split_theta_to_array(self.ml_params)

            popt, cov_popt, info, msg, status = leastsq(residuals,
                    x0=initial_params,
                    full_output=True)
                #Dfun=,
                #col_deriv=True)
            
            if np.any(np.isnan(info["fjac"])):
                return False

            leastsq_success_statuses = [1,2,3,4]
            if status in leastsq_success_statuses:
                if any(np.isnan(popt)):
                    return False
                #within bounds?
                if not self.are_params_in_bounds(popt):
                    return False

                sigma = self.ml_sigma(x, y, popt, weights)
                self.ml_params = np.append(popt, [sigma])

                logging.info("leastsq successful for model %s" % self.function.__name__)

                return True
            else:
                logging.warn("leastsq NOT successful for model %s, msg: %s" % (self.function.__name__, msg))
                logging.warn("best parameters found: " + str(popt))
                return False
        except Exception as e:
            print(e)
            tb = traceback.format_exc()
            print(tb)
            return False

    def fit_bfgs(self, x, y, weights, start_from_default):
        try:
            def objective(params):
                if weights is None:
                    if self.recency_weighting:
                        return np.sum(recency_weights(len(y))*(self.function(x, *params) - y)**2)
                    else:
                        return np.sum((self.function(x, *params) - y)**2)
                else:
                    if self.recency_weighting:
                        return np.sum(weights * recency_weights(len(y)) * (self.function(x, *params) - y)**2)
                    else:
                        return np.sum(weights * (self.function(x, *params) - y)**2)
            bounds = []
            for param_name in self.function_params:
                if param_name in self.min_vals and param_name in self.max_vals:
                    bounds.append((self.min_vals[param_name], self.max_vals[param_name]))
                elif param_name in self.min_vals:
                    bounds.append((self.min_vals[param_name], None))
                elif param_name in self.max_vals:
                    bounds.append((None, self.max_vals[param_name]))
                else:
                    bounds.append((None, None))

            if start_from_default:
                initial_params = self.default_function_param_array()
            else:
                initial_params, _ = self.split_theta_to_array(self.ml_params)

            popt, fval, info= fmin_l_bfgs_b(objective,
                                            x0=initial_params,
                                            bounds=bounds,
                                            approx_grad=True)
            if info["warnflag"] != 0:
                logging.warn("BFGS not converged! (warnflag %d) for model %s" % (info["warnflag"], self.name))
                logging.warn(info)
                return False

            if popt is None:
                return False
            if any(np.isnan(popt)):
                logging.info("bfgs NOT successful for model %s, parameter NaN" % self.name)
                return False
            sigma = self.ml_sigma(x, y, popt, weights)
            self.ml_params = np.append(popt, [sigma])
            logging.info("bfgs successful for model %s" % self.name)
            return True
        except:
            return False

    def aic(self, x, y):
        """
            Akaike information criterion
            http://en.wikipedia.org/wiki/Akaike_information_criterion
        """
        params, sigma = self.split_theta_to_array(self.ml_params)
        y_model = self.function(x, *params)
        log_likelihood = norm.logpdf(y-y_model, loc=0, scale=sigma).sum()
        return 2 * len(self.function_params) - 2 * log_likelihood



class MCMCCurveModel(CurveModel):
    """
        MLE curve fitting + MCMC sampling with uniform priors for parameter uncertainty.

        Model: y ~ f(x) + eps with eps ~ N(0, sigma^2)
    """
    def __init__(self,
                 function,
                 function_der=None,
                 min_vals={},
                 max_vals={},
                 default_vals={},
                 burn_in=300,
                 nwalkers=100,
                 nsamples=800,
                 nthreads=1,
                 recency_weighting=False):
        """
            function: the function to be fit
            function_der: derivative of that function
        """
        super(MCMCCurveModel, self).__init__(
            function=function,
            function_der=function_der,
            min_vals=min_vals,
            max_vals=max_vals,
            default_vals=default_vals)
        self.ml_curve_model = MLCurveModel(
            function=function,
            function_der=function_der,
            min_vals=self.min_vals,
            max_vals=self.max_vals,
            default_vals=self.default_vals,
            recency_weighting=recency_weighting)

        #TODO: have two burn-ins, one for when the ML fitting is successful and one for when not!
        self.burn_in = burn_in
        self.nwalkers = nwalkers
        self.nsamples = nsamples
        self.nthreads = nthreads
        self.recency_weighting = recency_weighting
    
    def fit(self, x, y):
        try:
            if self.ml_curve_model.fit(x, y):
                logging.info("ML fit: " + str(self.ml_curve_model.ml_params))
                self.fit_mcmc(x, y)
                return True
            else:
                return False
        except Exception as e:
            print(e)
            tb = traceback.format_exc()
            print(tb)
            return False

    #priors
    def ln_prior(self, theta):
        """
            log-prior is (up to a constant)
        """
        if self.are_params_in_bounds(theta):
            return 0.0
        else:
            return -np.inf
    
    #likelihood
    def ln_likelihood(self, theta, x, y):
        """
           y = y_true + y_noise
            with y_noise ~ N(0, sigma^2)
        """
        params, sigma = self.split_theta(theta)
        y_model = self.function(x, **params)
        if self.recency_weighting:
            weight = recency_weights(len(y))
            ln_likelihood = (weight*norm.logpdf(y-y_model, loc=0, scale=sigma)).sum()
        else:
            ln_likelihood = norm.logpdf(y-y_model, loc=0, scale=sigma).sum()
        if np.isnan(ln_likelihood):
            return -np.inf
        else:
            return ln_likelihood
        
    def ln_prob(self, theta, x, y):
        """
            posterior probability
        """
        lp = self.ln_prior(theta)
        if not np.isfinite(lp):
            return -np.inf
        return lp + self.ln_likelihood(theta, x, y)
    
    def fit_mcmc(self, x, y):
        #initialize in an area around the starting position
        #pos = [start + 1e-4*np.random.randn(self.ndim) for i in range(self.nwalkers)]
        assert self.ml_curve_model.ml_params is not None
        pos = [self.ml_curve_model.ml_params + 1e-6*np.random.randn(self.ndim) for i in range(self.nwalkers)]
        if self.nthreads <= 1:
            sampler = emcee.EnsembleSampler(self.nwalkers,
                self.ndim,
                self.ln_prob,
                args=(x, y))
        else:
            sampler = emcee.EnsembleSampler(
                self.nwalkers,
                self.ndim,
                model_ln_prob,
                args=(self, x, y),
                threads=self.nthreads)
        sampler.run_mcmc(pos, self.nsamples)
        self.mcmc_chain = sampler.chain
        
    def get_burned_in_samples(self):
        samples = self.mcmc_chain[:, self.burn_in:, :].reshape((-1, self.ndim))
        return samples

    def predictive_distribution(self, x, thin=1):
        assert isinstance(x, float) or isinstance(x, int)
        samples = self.get_burned_in_samples()
        predictions = []
        for theta in samples[::thin]:
            params, sigma = self.split_theta(theta)
            predictions.append(self.function(x, **params))
        return np.asarray(predictions)

    def predictive_ln_prob_distribution(self, x, y, thin=1):
        """
            posterior log p(y|x,D) for each sample
        """
        #assert isinstance(x, float) or isinstance(x, int)
        samples = self.get_burned_in_samples()
        ln_probs = []
        for theta in samples[::thin]:
            ln_prob = self.ln_likelihood(theta, x, y)
            ln_probs.append(ln_prob)
        return np.asarray(ln_probs)

    def posterior_ln_prob(self, x, y, thin=10):
        """
            posterior log p(y|x,D)

            1/S sum p(y|D,theta_s)
            equivalent to:
            logsumexp(log p(y|D,theta_s)) - log(S)
        """
        assert not np.isscalar(x)
        assert not np.isscalar(y)
        x = np.asarray(x)
        y = np.asarray(y)
        ln_probs = self.predictive_ln_prob_distribution(x, y)
        #print ln_probs
        #print np.max(ln_probs)
        #print np.min(ln_probs)
        #print np.mean(ln_probs)
        #print "logsumexp(ln_probs)", logsumexp(ln_probs)
        #print "np.log(len(ln_probs)) ", np.log(len(ln_probs))
        #print logsumexp(ln_probs) - np.log(len(ln_probs))
        return logsumexp(ln_probs) - np.log(len(ln_probs))

    def predict(self, x):
        """
            E[f(x)]
        """
        predictions = self.predictive_distribution(x)
        return np.ma.masked_invalid(predictions).mean()
    
    def predictive_density(self, x_pos, x_density):
        density = kde.gaussian_kde(self.predictive_distribution(x_pos))
        return density(x_density)

    def prob_x_greater_than(self, x, y, theta):
        """
            P(f(x) > y | Data, theta)
        """
        params, sigma = self.split_theta(theta)
        mu = self.function(x, **params)
        cdf = norm.cdf(y, loc=mu, scale=sigma)
        return 1. - cdf

    def posterior_mean_prob_x_greater_than(self, x, y, thin=1):
        """
            P(E[f(x)] > E[y] | Data)

            thin: only use every thin'th sample
        
            Posterior probability that the expected valuef(x) is greater than 
            the expected value of y.
        """
        posterior_distribution = self.predictive_distribution(x, thin)
        return masked_mean_x_greater_than(posterior_distribution, y)


    def posterior_prob_x_greater_than(self, x, y, thin=1):
        """
            P(f(x) > y | Data)
        
            Posterior probability that f(x) is greater than y.
        """
        assert isinstance(x, float) or isinstance(x, int)
        assert isinstance(y, float) or isinstance(y, int)
        probs = []
        samples = self.get_burned_in_samples()
        for theta in samples[::thin]:
            probs.append(self.prob_x_greater_than(x, y, theta))

        return np.ma.masked_invalid(probs).mean()

    def posterior_log_likelihoods(self, x, y):
        #DEPRECATED!
        samples = self.get_burned_in_samples()
        log_likelihoods = []
        for theta in samples:
            params, sigma = self.split_theta(theta)
            log_likelihood = self.ln_likelihood(theta, x, y)
            #TODO: rather add a -np.inf?
            if not np.isnan(log_likelihood) and np.isfinite(log_likelihood):
                log_likelihoods.append(log_likelihood)
        return log_likelihoods

    def mean_posterior_log_likelihood(self, x, y):
        #DEPRECATED!
        return np.ma.masked_invalid(self.posterior_log_likelihoods(x, y)).mean()

    def median_posterior_log_likelihood(self, x, y):
        #DEPRECATED!
        masked_x = np.ma.masked_invalid(self.posterior_log_likelihoods(x, y))
        return np.ma.extras.median(masked_x)

    def max_posterior_log_likelihood(self, x, y):
        #DEPRECATED!
        return np.ma.masked_invalid(self.posterior_log_likelihoods(x, y)).max()

    def posterior_log_likelihood(self, x, y):
        #DEPRECATED!
        return self.median_posterior_log_likelihood(x, y)

    def predictive_std(self, x, thin=1):
        """
           sqrt(Var[f(x)])
        """
        predictions = self.predictive_distribution(x, thin)
        return np.ma.masked_invalid(predictions).std()

    def dic(self, x, y):
        """ Deviance Information Criterion. """
        samples = self.get_burned_in_samples()
        deviances = []
        for theta in samples:
            params, sigma = self.split_theta(theta)
            deviance = -2 * self.ln_likelihood(theta, x, y)
            deviances.append(deviance)
        mean_theta = samples.mean(axis=0)
        theta_mean_deviance = -2 * self.ln_likelihood(mean_theta, x, y)
        DIC = 2 * np.mean(deviances) - theta_mean_deviance
        return DIC


class LinearCurveModel(CurveModel):
    """
        Fits a function f(x) = a * x + b using OLS.
    """

    def __init__(self, *arg, **kwargs):
        if "default_vals" in kwargs:
            logging.warn("default values not needed for the linear model.")
        kwargs["default_vals"] = {"a": 0, "b": 0}
        kwargs["min_vals"] = {"a": 0}
        super(LinearCurveModel, self).__init__(
                function=all_models["linear"],
                *arg,
                **kwargs)

    def fit(self, x, y, weights=None, start_from_default=True):
        return self.fit_ml(x, y, weights)

    def fit_ml(self, x, y, weights):
        """
            Ordinary Least Squares fit.

            TODO: use the weights!
        """
        #TODO: check if the results agree with the minimum/maximum values!
        X = np.asarray([np.ones(len(x)), x]).T
        bh = np.dot(np.linalg.inv(np.dot(X.T,X)),np.dot(X.T,y))
        a = bh[1]
        b = bh[0]
        sigma = (y-self.function(x, a, b)).std()
        self.ml_params = np.asarray([a, b, sigma])
        return True

    def predict(self, x):
        a = self.ml_params[0]
        b = self.ml_params[1]
        return a * x + b


class LinearMCMCCurveModel(MCMCCurveModel):
    def __init__(self, **kwargs):
        ml_curve_model = LinearCurveModel()
        super(LinearMCMCCurveModel, self).__init__(
            function=ml_curve_model.function,
            min_vals=ml_curve_model.min_vals,
            max_vals=ml_curve_model.max_vals,
            default_vals=ml_curve_model.default_vals,
            **kwargs)
        self.ml_curve_model = ml_curve_model


def model_ln_prob(theta, model, x, y):
    return model.ln_prob(theta, x, y)


class MCMCCurveModelCombination(object):

    def __init__(self,
            ml_curve_models,
            xlim,
            burn_in=500,
            nwalkers=100,
            nsamples=2500,
            normalize_weights=True,
            monotonicity_constraint=True,
            soft_monotonicity_constraint=False,
            initial_model_weight_ml_estimate=False,
            normalized_weights_initialization="constant",
            strictly_positive_weights=True,
            sanity_check_prior=True,
            nthreads=1,
            recency_weighting=True):
        """
            xlim: the point on the x axis we eventually want to make predictions for.
        """
        self.ml_curve_models = ml_curve_models
        self.xlim = xlim
        self.burn_in = burn_in
        self.nwalkers = nwalkers
        self.nsamples = nsamples
        self.normalize_weights = normalize_weights
        assert not (monotonicity_constraint and soft_monotonicity_constraint), "choose either the monotonicity_constraint or the soft_monotonicity_constraint, but not both"
        self.monotonicity_constraint = monotonicity_constraint
        self.soft_monotonicity_constraint = soft_monotonicity_constraint
        self.initial_model_weight_ml_estimate = initial_model_weight_ml_estimate
        self.normalized_weights_initialization = normalized_weights_initialization
        self.strictly_positive_weights = strictly_positive_weights
        self.sanity_check_prior = sanity_check_prior
        self.nthreads = nthreads
        self.recency_weighting = recency_weighting
        #the constant used for initializing the parameters in a ball around the ML parameters
        self.rand_init_ball = 1e-6
        self.name = "model combination"# (%s)" % ", ".join([model.name for model in self.ml_curve_models])

    def fit(self, x, y, model_weights=None):
        if self.fit_ml_individual(x, y, model_weights):
            #run MCMC:
            self.fit_mcmc(x, y)
            return True
        else:
            print("fit_ml_individual failed")
            return False

    def y_lim_sanity_check(self, ylim):
        # just make sure that the prediction is not below 0 nor insanely big
        # HOWEVER: there might be cases where some models might predict value larger than 1.0
        # and this is alright, because in those cases we don't necessarily want to stop a run.
        if not np.isfinite(ylim) or ylim < 0. or ylim > 100.0:
            return False
        else:
            return True

    def fit_ml_individual(self, x, y, model_weights):
        """
            Do a ML fit for each model individually and then another ML fit for the combination of models.
        """
        self.fit_models = []
        for model in self.ml_curve_models:
            if model.fit(x, y):
                ylim = model.predict(self.xlim)
                if not self.y_lim_sanity_check(ylim):
                    print("ML fit of model %s is out of bound range [0.0, 100.] at xlim." % (model.function.__name__))
                    continue
                params, sigma = model.split_theta_to_array(model.ml_params)
                if not np.isfinite(self.ln_model_prior(model, params)):
                    print("ML fit of model %s is not supported by prior." % model.function.__name__)
                    continue
                self.fit_models.append(model)
                    
        if len(self.fit_models) == 0:
            return False

        if model_weights is None:
            if self.normalize_weights:
                if self.normalized_weights_initialization == "constant":
                    #initialize with a constant value
                    #we will sample in this unnormalized space and then later normalize
                    model_weights = [10. for model in self.fit_models]
                else:# self.normalized_weights_initialization == "normalized"
                    model_weights = [1./len(self.fit_models) for model in self.fit_models]
            else:
                if self.initial_model_weight_ml_estimate:
                    model_weights = self.get_ml_model_weights(x, y)
                    print(model_weights)
                    non_zero_fit_models = []
                    non_zero_weights = []
                    for w, model in zip(model_weights, self.fit_models):
                        if w > 1e-4:
                            non_zero_fit_models.append(model)
                            non_zero_weights.append(w)
                    self.fit_models = non_zero_fit_models
                    model_weights = non_zero_weights
                else:
                    model_weights = [1./len(self.fit_models) for model in self.fit_models]

        #build joint ml estimated parameter vector
        model_params = []
        all_model_params = []
        for model in self.fit_models:
            params, sigma = model.split_theta_to_array(model.ml_params)
            model_params.append(params)
            all_model_params.extend(params)

        y_predicted = self.predict_given_params(x, model_params, model_weights)
        sigma = (y - y_predicted).std()

        self.ml_params = self.join_theta(all_model_params, sigma, model_weights)
        self.ndim = len(self.ml_params)
        if self.nwalkers < 2*self.ndim:
            self.nwalkers = 2*self.ndim
            print("warning: increasing number of walkers to 2*ndim=%d" % (self.nwalkers))
        return True


    def get_ml_model_weights(self, x, y_target):
        """
            Get the ML estimate of the model weights.
        """

        """
            Take all the models that have been fit using ML.
            For each model we get a prediction of y: y_i

            Now how can we combine those to reduce the squared error:

                argmin_w (y_target - w_1 * y_1 - w_2 * y_2 - w_3 * y_3 ...)

            Deriving and setting to zero we get a linear system of equations that we need to solve.


            Resource on QP:
            http://stats.stackexchange.com/questions/21565/how-do-i-fit-a-constrained-regression-in-r-so-that-coefficients-total-1
            http://maggotroot.blogspot.de/2013/11/constrained-linear-least-squares-in.html
        """
        num_models = len(self.fit_models)
        y_predicted = []
        b = []
        for model in self.fit_models:
            y_model = model.predict(x)
            y_predicted.append(y_model)
            b.append(y_model.dot(y_target))
        a = np.zeros((num_models, num_models))
        for i in range(num_models):
            for j in range(num_models):
                a[i, j] = y_predicted[i].dot(y_predicted[j])
                #if i == j:
                #    a[i, j] -= 0.1 #constraint the weights!
        a_rank = np.linalg.matrix_rank(a)
        if a_rank != num_models:
            print("Rank %d not sufficcient for solving the linear system. %d needed at least." % (a_rank, num_models))
        try:
            print(np.linalg.lstsq(a, b)[0])
            print(np.linalg.solve(a, b))
            print(nnls(a, b)[0])
            ##return np.linalg.solve(a, b)
            weights = nnls(a, b)[0]
            #weights = [w if w > 1e-4 else 1e-4 for w in weights]
            return weights
        #except LinAlgError as e:
        except:
            return [1./len(self.fit_models) for model in self.fit_models]


    #priors
    def ln_prior(self, theta):
        ln = 0
        model_params, sigma, model_weights = self.split_theta(theta)
        for model, params in zip(self.fit_models, model_params):
            ln += self.ln_model_prior(model, params)
        #if self.normalize_weights:
            #when we normalize we expect all weights to be positive
        #we expect all weights to be positive
        if self.strictly_positive_weights and np.any(model_weights < 0):
            return -np.inf
        return ln


    def ln_model_prior(self, model, params):
        if not model.are_params_in_bounds(params):
            return -np.inf
        if self.monotonicity_constraint:
            #check for monotonicity(this obviously this is a hack, but it works for now):
            x_mon = np.linspace(2, self.xlim, 100)
            y_mon = model.function(x_mon, *params)
            if np.any(np.diff(y_mon) < 0):
                return -np.inf
        elif self.soft_monotonicity_constraint:
            #soft monotonicity: defined as the last value being bigger than the first one
            x_mon = np.asarray([2, self.xlim])
            y_mon = model.function(x_mon, *params)
            if y_mon[0] > y_mon[-1]:
                return -np.inf
        ylim = model.function(self.xlim, *params)
        #sanity check for ylim
        if self.sanity_check_prior and not self.y_lim_sanity_check(ylim):
            return -np.inf
        else:
            return 0.0

    #likelihood
    def ln_likelihood(self, theta, x, y):
        y_model, sigma = self.predict_given_theta(x, theta)

        # We used our own logpdf function here
        if self.recency_weighting:
            weight = recency_weights(len(y))
            ln_likelihood = (weight*norm.logpdf(y-y_model, loc=0, scale=sigma)).sum()
            #ln_likelihood = (weight * logpdf(y=y-y_model, loc=0, scale=sigma)).sum()
        else:
            ln_likelihood = norm.logpdf(y-y_model, loc=0, scale=sigma).sum()
            #ln_likelihood = logpdf(y=y-y_model, loc=0, scale=sigma).sum()

        if np.isnan(ln_likelihood):
            return -np.inf
        else:
            return ln_likelihood

    def ln_prob(self, theta, x, y):
        """
            posterior probability
        """
        lp = self.ln_prior(theta)
        if not np.isfinite(lp):
            return -np.inf
        return lp + self.ln_likelihood(theta, x, y)

    def split_theta(self, theta):
        """
            theta is structured as follows:
            for each model i
                for each model parameter j
            theta = (theta_ij, sigma, w_i)
        """
        num_models = len(self.fit_models)

        model_weights = theta[-len(self.fit_models):]

        all_model_params = []
        for model in self.fit_models:
            num_model_params = len(model.function_params)
            model_params = theta[:num_model_params]
            all_model_params.append(model_params)

            theta = theta[num_model_params:]
        sigma = theta[0]
        model_weights = theta[1:]
        assert len(model_weights) == len(self.fit_models)
        return all_model_params, sigma, model_weights


    def join_theta(self, model_params, sigma, model_weights):
        #assert len(model_params) == len(model_weights)
        theta = []
        theta.extend(model_params)
        theta.append(sigma)
        theta.extend(model_weights)
        return theta

    def fit_mcmc(self, x, y):
        #initialize in an area around the starting position

        assert self.ml_params is not None
        pos = [self.ml_params + self.rand_init_ball*np.random.randn(self.ndim) for i in range(self.nwalkers)]

        if self.nthreads <= 1:
            sampler = emcee.EnsembleSampler(self.nwalkers,
                self.ndim,
                self.ln_prob,
                args=(x, y))
        else:
            sampler = emcee.EnsembleSampler(
                self.nwalkers,
                self.ndim,
                model_ln_prob,
                args=(self, x, y),
                threads=self.nthreads)
        sampler.run_mcmc(pos, self.nsamples)
        self.mcmc_chain = sampler.chain

        if self.normalize_weights:
            self.normalize_chain_model_weights()

    def normalize_chain_model_weights(self):
        """
            In the chain we sample w_1,... w_i however we are interested in the model
            probabilities p_1,... p_i
        """
        model_weights_chain = self.mcmc_chain[:,:,-len(self.fit_models):]
        model_probabilities_chain = model_weights_chain / model_weights_chain.sum(axis=2)[:,:,np.newaxis]
        #replace in chain
        self.mcmc_chain[:,:,-len(self.fit_models):] = model_probabilities_chain

    def get_burned_in_samples(self):
        samples = self.mcmc_chain[:, self.burn_in:, :].reshape((-1, self.ndim))
        return samples

    def print_probs(self):
        burned_in_chain = self.get_burned_in_samples()
        model_probabilities = burned_in_chain[:,-len(self.fit_models):]
        print(model_probabilities.mean(axis=0))

    def predict_given_theta(self, x, theta):
        """
            returns y_predicted, sigma
        """
        model_params, sigma, model_weights = self.split_theta(theta)

        y_predicted = self.predict_given_params(x, model_params, model_weights)

        return y_predicted, sigma

    def predict_given_params(self, x, model_params, model_weights):
        """
            returns y_predicted
        """
        if self.normalize_weights:
            model_weight_sum = np.sum(model_weights)
            model_ws = [weight/model_weight_sum for weight in model_weights]
        else:
            model_ws = model_weights

        y_model = []
        for model, model_w, params in zip(self.fit_models, model_ws, model_params):
            y_model.append(model_w*model.function(x, *params))
        y_predicted = reduce(lambda a, b: a+b, y_model)
        return y_predicted

    def prob_x_greater_than(self, x, y, theta):
        """
            P(f(x) > y | Data, theta)
        """
        model_params, sigma, model_weights = self.split_theta(theta)

        y_predicted = self.predict_given_params(x, model_params, model_weights)

        # Replaced with our cdf function
        cdf = norm.cdf(y, loc=y_predicted, scale=sigma)
        #c = cdf(y, loc=0, scale=sigma)
        return 1. - cdf

    def posterior_prob_x_greater_than(self, x, y, thin=1):
        """
            P(f(x) > y | Data)

            thin: only use every thin'th sample
        
            Posterior probability that f(x) is greater than y.
        """
        assert isinstance(x, float) or isinstance(x, int)
        assert isinstance(y, float) or isinstance(y, int)
        probs = []
        samples = self.get_burned_in_samples()
        for theta in samples[::thin]:   
            probs.append(self.prob_x_greater_than(x, y, theta))
        return np.ma.masked_invalid(probs).mean()


    def posterior_mean_prob_x_greater_than(self, x, y, thin=1):
        """
            P(E[f(x)] > E[y] | Data)

            thin: only use every thin'th sample
        
            Posterior probability that the expected valuef(x) is greater than 
            the expected value of y.
        """
        posterior_distribution = self.predictive_distribution(x, thin)
        return masked_mean_x_greater_than(posterior_distribution, y)


    def predictive_distribution(self, x, thin=1):
        assert isinstance(x, float) or isinstance(x, int)
        samples = self.get_burned_in_samples()
        predictions = []
        for theta in samples[::thin]:
            model_params, sigma, model_weights = self.split_theta(theta)
            y_predicted = self.predict_given_params(x, model_params, model_weights)
            predictions.append(y_predicted)
        return np.asarray(predictions)

    def predictive_ln_prob_distribution(self, x, y, thin=1):
        """
            posterior log p(y|x,D) for each sample
        """
        #assert isinstance(x, float) or isinstance(x, int)
        samples = self.get_burned_in_samples()
        ln_probs = []
        for theta in samples[::thin]:
            ln_prob = self.ln_likelihood(theta, x, y)
            ln_probs.append(ln_prob)
        return np.asarray(ln_probs)

    def posterior_ln_prob(self, x, y, thin=10):
        """
            posterior log p(y|x,D)

            1/S sum p(y|D,theta_s)
            equivalent to:
            logsumexp(log p(y|D,theta_s)) - log(S)
        """
        assert not np.isscalar(x)
        assert not np.isscalar(y)
        x = np.asarray(x)
        y = np.asarray(y)
        ln_probs = self.predictive_ln_prob_distribution(x, y)
        return logsumexp(ln_probs) - np.log(len(ln_probs))

    def predict(self, x, thin=1):
        """
            E[f(x)]
        """
        predictions = self.predictive_distribution(x, thin)
        return np.ma.masked_invalid(predictions).mean()

    def predictive_std(self, x, thin=1):
        """
           sqrt(Var[f(x)])
        """
        predictions = self.predictive_distribution(x, thin)
        return np.ma.masked_invalid(predictions).std()

    def serialize(self, fname):
        import pickle
        pickle.dump(self, open(fname, "wb"))
class MlCurveMixtureModel(object):
    """
        Maximum Likelihood fit of a convex combination of curve models
        using the Expectation Maxization algorithm.

        http://www.slideshare.net/butest/lecture-18-gaussian-mixture-models-and-expectation-maximization
        http://melodi.ee.washington.edu/people/bilmes/mypapers/em.pdf
        http://www.igi.tugraz.at/lehre/MLA/WS07/chapter9.pdf

        With Dirichlet prior:
            ftp://tlp.limsi.fr/public/map93.pdf

        Finite Mixture Model with Dirichlet Distribution
            http://blog.datumbox.com/finite-mixture-model-based-on-dirichlet-distribution/

        Variational Bayesian Gaussian Mixture Model (VBGMM)
            http://kittipatkampa.wordpress.com/2010/10/14/variational-bayesian-gaussian-mixture-model-vbgmm/
    """
    def __init__(self, ml_curve_models):
        self.ml_curve_models = ml_curve_models

    def fit(self, x, y, num_iter=1):
        fit_models = []
        for model in self.ml_curve_models:
            if model.fit(x, y, start_from_default=True):
                fit_models.append(model)
        model_weights = [1./len(fit_models) for m in fit_models]
        if len(fit_models) == 0:
            return False
        try:
            for i in range(0, num_iter):
                #E-step:
                responsibilities = []
                for model_weight, model in zip(model_weights, fit_models):
                    #responsibilities.append(0.000001 + model_weight * model.likelihood(x, y))
                    #responsibilities.append(0.0001 + model_weight * model.likelihood(x, y))
                    responsibilities.append(0.000001 + model_weight * model.likelihood(x, y))
                responsibilities = np.asarray(responsibilities)
                #normalize:
                responsibilities = responsibilities / responsibilities.sum(axis=0)

                #M-step:
                previous_fit_model_weights = responsibilities.mean(axis=1)
                new_fit_models = []
                model_weights = []
                for model_idx, model in enumerate(fit_models):
                    if (previous_fit_model_weights[model_idx] > 0.000001
                        and model.fit(x, y, responsibilities[model_idx, :], start_from_default=False)):
                        new_fit_models.append(model)
                        #model_weights.append(previous_fit_model_weights[model_idx])
                        model_weights.append(0.01 + previous_fit_model_weights[model_idx])
                model_weights = np.asarray(model_weights)
                #renormalize (in case a model couldn't be fit anymore)
                model_weights = model_weights / model_weights.sum()
                fit_models = new_fit_models
                for model_weight, model in zip(model_weights, fit_models):
                    logging.debug("%s %f" % (model.function.__name__, model_weight))
            
            #print model_weights
            self.model_weights = model_weights
            self.fit_models = fit_models
            return True
        except: 
            return False

    def predict(self, x):
        y_predicted = None
        for model_weight, model in zip(self.model_weights, self.fit_models):
            if y_predicted is None:
                y_predicted = model_weight * model.predict(x)
            else:
                y_predicted += model_weight * model.predict(x)
        return y_predicted


class MCMCCurveMixtureModel(object):

    def __init__(self,
            ml_curve_models,
            xlim,
            burn_in=600,
            nwalkers=80,
            nsamples=5000,
            monotonicity_constraint=True,
            soft_monotonicity_constraint=False,
            nthreads=1,
            recency_weighting=True):
        """
            xlim: the point on the x axis we eventually want to make predictions for.
        """
        self.ml_curve_models = ml_curve_models
        self.ml_curve_mixture_model = MlCurveMixtureModel(ml_curve_models)
        self.xlim = xlim
        self.burn_in = burn_in
        self.nwalkers = nwalkers
        self.nsamples = nsamples
        assert not (monotonicity_constraint and soft_monotonicity_constraint), "choose either the monotonicity_constraint or the soft_monotonicity_constraint, but not both"
        self.monotonicity_constraint = monotonicity_constraint
        self.soft_monotonicity_constraint = soft_monotonicity_constraint
        self.nthreads = nthreads
        self.recency_weighting = recency_weighting
        #the constant used for initializing the parameters in a ball around the ML parameters
        self.rand_init_ball = 1e-6

    def fit(self, x, y):
        if self.fit_ml_individual(x, y):
            #run MCMC:
            self.fit_mcmc(x, y)
            return True
        else:
            print("fit_ml_individual failed")
            return False

    def fit_ml_individual(self, x, y):
        """
            Do a ML fit for each model individually and then another ML fit for the combination of models.
        """
        if self.ml_curve_mixture_model.fit(x, y):
            model_weights = self.ml_curve_mixture_model.model_weights
            self.fit_models = self.ml_curve_mixture_model.fit_models
        else:
            self.fit_models = []
            for model in self.ml_curve_models:
                if model.fit(x, y):
                    if np.isfinite(self.ln_model_prior(model, model.ml_params)):
                        self.fit_models.append(model)
                    else:
                        print("ML fit of model %s is not supported by prior." % model.function.__name__)
            model_weights = [10. for model in self.fit_models]
            if len(self.fit_models) == 0:
                return False

        #build joint ml estimated parameter vector
        all_model_params = []
        for model in self.fit_models:
            all_model_params.extend(model.ml_params)
        print("model weights: ", model_weights)
        self.ml_params = self.join_theta(all_model_params, model_weights)
        self.ndim = len(self.ml_params)
        return True

    #priors
    def ln_prior(self, theta):
        ln = 0
        model_thetas, model_weights = self.split_theta(theta)
        for model, theta in zip(self.fit_models, model_thetas):
            ln += self.ln_model_prior(model, theta)
        #if self.normalize_weights:
            #when we normalize we expect all weights to be positive
        #we expect all weights to be positive
        if np.any(model_weights < 0):
            return -np.inf
        return ln

    def ln_model_prior(self, model, theta):
        if not model.are_params_in_bounds(theta):
            return -np.inf
        if self.monotonicity_constraint:
            #check for monotonicity(this obviously this is a hack, but it works for now):
            x_mon = np.linspace(2, self.xlim, 100)
            params, sigma = model.split_theta_to_array(theta)
            y_mon = model.function(x_mon, *params)
            if np.any(np.diff(y_mon) < 0):
                return -np.inf
        elif self.soft_monotonicity_constraint:
            #soft monotonicity: defined as the last value being bigger than the first one
            x_mon = np.asarray([2, self.xlim])
            y_mon = model.function(x_mon, *params)
            if y_mon[0] > y_mon[-1]:
                return -np.inf
        return 0.0

    #likelihood
    def ln_likelihood(self, theta, x, y):
        """
        """
        sample_weights = None
        if self.recency_weighting:
            sample_weights = [10**(1./len(y))] * len(y)
            sample_weights = sample_weights**(np.arange(0, len(y)))

        model_thetas, model_weights = self.split_theta(theta)
        #normalize the weights
        model_weight_sum = np.sum(model_weights)
        model_weights = [weight/model_weight_sum for weight in model_weights]

        ln_likelihoods = []
        for model, model_theta, model_weight in zip(self.fit_models, model_thetas, model_weights):
            ln_likelihood = np.log(model_weight)
            params, sigma = model.split_theta_to_array(model_theta)
            y_model = model.function(x, *params)
            if sample_weights is None:
                ln_likelihood += norm.logpdf(y-y_model, loc=0, scale=sigma).sum()
            else:
                ln_likelihood += (sample_weights*norm.logpdf(y-y_model, loc=0, scale=sigma)).sum()

            ln_likelihoods.append(ln_likelihood)

        if np.any(np.isnan(ln_likelihoods)):
            return -np.inf
        else:
            return logsumexp(ln_likelihoods)

    def ln_prob(self, theta, x, y):
        """
            posterior probability
        """
        lp = self.ln_prior(theta)
        if not np.isfinite(lp):
            return -np.inf
        return lp + self.ln_likelihood(theta, x, y)

    def split_theta(self, theta):
        """
            theta is structured as follows:
            for each model i
                for each model parameter j
            theta = (theta_ij, sigma, w_i)
        """
        num_models = len(self.fit_models)

        model_weights = theta[-len(self.fit_models):]

        all_model_params = []
        for model in self.fit_models:
            num_model_params = len(model.all_param_names)
            model_params = theta[:num_model_params]
            all_model_params.append(model_params)

            theta = theta[num_model_params:]
        model_weights = theta
        assert len(model_weights) == len(self.fit_models)
        return all_model_params, model_weights


    def join_theta(self, model_params, model_weights):
        #assert len(model_params) == len(model_weights)
        theta = []
        theta.extend(model_params)
        theta.extend(model_weights)
        return theta

    def fit_mcmc(self, x, y):
        #initialize in an area around the starting position

        assert self.ml_params is not None
        pos = [self.ml_params + self.rand_init_ball*np.random.randn(self.ndim) for i in range(self.nwalkers)]

        if self.nthreads <= 1:
            sampler = emcee.EnsembleSampler(self.nwalkers,
                self.ndim,
                self.ln_prob,
                args=(x, y))
        else:
            sampler = emcee.EnsembleSampler(
                self.nwalkers,
                self.ndim,
                model_ln_prob,
                args=(self, x, y),
                threads=self.nthreads)
        sampler.run_mcmc(pos, self.nsamples)
        self.mcmc_chain = sampler.chain

        #we normalize the weights in the chain model, so that the trace plot make more sense
        self.normalize_chain_model_weights()

    def normalize_chain_model_weights(self):
        """
            In the chain we sample w_1,... w_i however we are interested in the model
            probabilities p_1,... p_i
        """
        model_weights_chain = self.mcmc_chain[:,:,-len(self.fit_models):]
        model_probabilities_chain = model_weights_chain / model_weights_chain.sum(axis=2)[:,:,np.newaxis]
        #replace in chain
        self.mcmc_chain[:,:,-len(self.fit_models):] = model_probabilities_chain

    def get_burned_in_samples(self):
        samples = self.mcmc_chain[:, self.burn_in:, :].reshape((-1, self.ndim))
        return samples

    def print_probs(self):
        burned_in_chain = self.get_burned_in_samples()
        model_probabilities = burned_in_chain[:,-len(self.fit_models):]
        print(model_probabilities.mean(axis=0))

    def predict_given_theta(self, x, theta):
        """
            returns y_predicted, sigma
        """
        model_params, model_weights = self.split_theta(theta)

        y_predicted = self.predict_given_params(x, model_params, model_weights)

        return y_predicted

    def predict_given_params(self, x, model_thetas, model_weights):
        """
            returns y_predicted
        """
        #normalize the weights
        model_weight_sum = np.sum(model_weights)
        model_ws = [weight/model_weight_sum for weight in model_weights]

        y_model = []
        for model, model_w, theta in zip(self.fit_models, model_ws, model_thetas):
            params, sigma = model.split_theta_to_array(theta)
            y_model.append(model_w*model.function(x, *params))
        y_predicted = reduce(lambda a, b: a+b, y_model)
        return y_predicted

    def prob_x_greater_than(self, x, y, theta):
        """
            P(f(x) > y | Data, theta)
        """
        model_params, model_weights = self.split_theta(theta)

        y_predicted = self.predict_given_params(x, model_params, model_weights)

        cdf = norm.cdf(y, loc=y_predicted, scale=sigma)
        return 1. - cdf

    def posterior_prob_x_greater_than(self, x, y, thin=1):
        """
            P(f(x) > y | Data)

            thin: only use every thin'th sample
        
            Posterior probability that f(x) is greater than y.
        """
        assert isinstance(x, float) or isinstance(x, int)
        assert isinstance(y, float) or isinstance(y, int)
        probs = []
        samples = self.get_burned_in_samples()
        for theta in samples[::thin]:   
            probs.append(self.prob_x_greater_than(x, y, theta))

        return np.ma.masked_invalid(probs).mean()

    def predictive_distribution(self, x, thin=1):
        assert isinstance(x, float) or isinstance(x, int)
        samples = self.get_burned_in_samples()
        predictions = []
        for theta in samples[::thin]:
            model_params, sigma, model_weights = self.split_theta(theta)
            y_predicted = self.predict_given_params(x, model_params, model_weights)
            predictions.append(y_predicted)
        return np.asarray(predictions)

    def predict(self, x, thin=1):
        """
            E[f(x)]
        """
        predictions = self.predictive_distribution(x, thin)
        return np.ma.masked_invalid(predictions).mean()

    def predictive_std(self, x, thin=1):
        """
            sqrt(Var[f(x)])
        """
        predictions = self.predictive_distribution(x, thin)
        return np.ma.masked_invalid(predictions).std()

    def serialize(self, fname):
        import pickle
        pickle.dump(self, open(fname, "wb"))


**Ensemble Curve Model**

In [7]:
def fit_model(model, x_train, y_train):
    success = model.fit(x_train, y_train)
    return (success, model)

def model_log_likelihood(model, x_test, y_test):
    return model.posterior_log_likelihood(x_test, y_test)

def model_posterior_prob_x_greater_than(model, x, y):
    return model.posterior_prob_x_greater_than(x, y)

def train_test_split(x, y, train_fraction):
    #split into train/test
    if train_fraction > 0.99:
        x_train = x
        y_train = y
        x_test = x
        y_test = y
    else:
        num_train = int(train_fraction * len(x))
        x_train = x[:num_train]
        y_train = y[:num_train]
        x_test = x[num_train:]
        y_test = y[num_train:]

    return x_train, y_train, x_test, y_test


class Ensemble(object):
    """
    """

    def __init__(self, models, map=map):
        """
            models: ensemble models
            map: map function, if multiprocessing is desired
        """
        self.all_models = models
        self._map = map
        self.fit_models = []


class CurveModelEnsemble(Ensemble):
    """

    """

    def __init__(self, models, map=map):
        super(CurveModelEnsemble, self).__init__(models, map)

    def fit(self, x, y, train_fraction=0.8):
        assert len(x) == len(y)

        model_log_likelihoods = []
        self.fit_models = []

        x_train, y_train, x_test, y_test = train_test_split(x, y, train_fraction)

        fit_result = self._map(
            partial(fit_model, x_train=x_train, y_train=y_train),
            self.all_models)
        for success, model in fit_result:
            if success:
                self.fit_models.append(model)

        if len(self.fit_models) == 0:
            logging.warn("EnsembleCurveModel couldn't fit any models!!!")
            return False

        model_log_likelihoods = self._map(
            partial(model_log_likelihood, x_test=x_test, y_test=y_test),
            self.fit_models)

        normalizing_constant = logsumexp(model_log_likelihoods)

        self.model_probabilities = [np.exp(log_lik - normalizing_constant) for log_lik in model_log_likelihoods]
        return True

    def posterior_prob_x_greater_than(self, x, y):
        """
            The probability under the models that a value y is exceeded at position x.
            IMPORTANT: if all models fail, by definition the probability is 1.0 and NOT 0.0
        """
        if len(self.fit_models) == 0:
            return 1.0

        models_prob_x_greater_than = model_log_likelihoods = self._map(
            partial(model_posterior_prob_x_greater_than, x=x, y=y),
            self.fit_models)

        overall_prob = 0
        for prob_x_greater_than, model_prob in zip(models_prob_x_greater_than, self.model_probabilities):
            overall_prob += model_prob * prob_x_greater_than
        return overall_prob

    def posterior_log_likelihood(self, x, y):
        log_liks = []
        for model, model_prob in zip(self.fit_models, self.model_probabilities):
            log_lik = model.posterior_log_likelihood(x, y)
            log_liks.append(np.log(model_prob) + log_lik)
        return logsumexp(log_liks)

    def predict(self, x):
        if np.isscalar(x):
            y = 0
        else:
            y = np.zeros(x.shape)
        for model, model_prob in zip(self.fit_models, self.model_probabilities):
            y += model_prob * model.predict(x)
        return y

    def __str__(self):
        ret = []
        model_names = [model.function.__name__ for model in self.fit_models]
        for model_prob, model_name in zip(self.model_probabilities, model_names):
            ret.append("%s: %f\n" % (model_name, model_prob))
        return "".join(ret)

    def serialize(self, fname):
        import pickle
        map_tmp = self._map
        self._map = None
        pickle.dump(self, open(fname, "wb"))
        self._map = map_tmp



class CurveEnsemble(Ensemble):
    """
        1. MCMC fitting
        2. Now take each theta as a model
        3. Make predictions as an weighted average of those models
            The weight is the (normalized) likelihood of some held out validation data.
    """

    def __init__(self, models, map=map):
        super(CurveEnsemble, self).__init__(models, map)

    def fit(self, x, y, train_fraction=0.8):
        assert len(x) == len(y)

        model_log_likelihoods = []
        self.fit_models = []

        x_train, y_train, x_test, y_test = train_test_split(x, y, train_fraction)

        fit_result = self._map(
            partial(fit_model, x_train=x_train, y_train=y_train),
            self.all_models)
        for success, model in fit_result:
            if success:
                self.fit_models.append(model)

        if len(self.fit_models) == 0:
            logging.warn("EnsembleCurveModel couldn't fit any models!!!")
            return

        #Now we interpret each theta as a separate model
        #TODO: parallelize!
        all_log_likelihoods = []
        for model in self.fit_models:
            model_log_likelihoods = []
            thetas = model.get_burned_in_samples()
            for theta_idx in range(thetas.shape[0]):
                theta = thetas[theta_idx,:]
                log_likelihood = model.ln_likelihood(theta, x_test, y_test)
                model_log_likelihoods.append(log_likelihood)
            all_log_likelihoods.append(model_log_likelihoods)

        self.model_theta_probabilities = [np.exp(model_log_likelihoods - logsumexp(model_log_likelihoods)) for model_log_likelihoods in all_log_likelihoods]

        normalizing_constant = logsumexp(all_log_likelihoods)

        normalize = lambda log_lik: np.exp(log_lik - normalizing_constant)

        self.all_model_probabilities = [[normalize(log_lik) for log_lik in model_log_likelihoods] for model_log_likelihoods in all_log_likelihoods]

        #sum up on a per model family basis:
        self.model_probabilities = [sum(model_probabilities) for model_probabilities in self.all_model_probabilities]


    def posterior_prob_x_greater_than(self, x, y):
        """
            The probability under the models that a value y is exceeded at position x.
            IMPORTANT: if all models fail, by definition the probability is 1.0 and NOT 0.0
        """
        if len(self.fit_models) == 0:
            return 1.0

        overall_prob = 0
        for model, theta_model_probabilities in zip(self.fit_models, self.all_model_probabilities):
            thetas = model.get_burned_in_samples()
            for theta_idx, theta_model_probability in zip(range(thetas.shape[0]), theta_model_probabilities):
                theta = thetas[theta_idx, :]
                overall_prob += theta_model_probability * model.prob_x_greater_than(x, y, theta)
        return overall_prob


    def predict(self, x):
        if np.isscalar(x):
            y = 0
        else:
            y = np.zeros(x.shape)
        #TOOD: implement!!
        return y

    def __str__(self):
        ret = []
        model_names = [model.function.__name__ for model in self.fit_models]
        for model_prob, model_name in zip(self.model_probabilities, model_names):
            ret.append("%s: %f\n" % (model_name, model_prob))
        return "".join(ret)

**MCMC Model Plotter**

In [8]:
def greek_label_mapping(oldlabels):
    labels = []
    for param_name in oldlabels:
        if param_name in ["alpha", "beta", "delta", "sigma"]:#"kappa", 
            labels.append("$\%s$" % param_name)
        else:
            labels.append("$%s$" % param_name)
    return labels

class MCMCCurveModelPlotter(object):
    def __init__(self, model):
        self.model = model
        
    def trace_plot(self,rasterized=False):
        num_plots = len(self.model.all_param_names)
        
        fig, axes = subplots(num_plots, 1, sharex=True, figsize=(8, 9))
        
        for idx, param_name in enumerate(self.model.all_param_names):
        
            axes[idx].plot(self.model.mcmc_chain[:, :, idx].T, color="k",
                alpha=0.4, rasterized=rasterized)
            axes[idx].yaxis.set_major_locator(MaxNLocator(5))
            #axes[0].axhline(m_true, color="#888888", lw=2)
            axes[idx].set_ylabel("$%s$" % param_name)
            
            if idx == num_plots-1:
                axes[idx].set_xlabel("step number")
        
        tight_layout(h_pad=0.0)
        

    def triangle_plot(self, labels=None, truths=None):
        samples = self.model.get_burned_in_samples()
        if labels is None:
            labels = greek_label_mapping(self.model.all_param_names)
        fig = triangle.corner(samples,
            labels=labels,
            truths=truths)

    def posterior_sample_plot(self, x, y=None, vline=None,
            xaxislabel="$x$", yaxislabel="$y$", alpha=0.3,
            label="", color="k", x_axis_scale=0.1, nsamples=50,
            plot_ml_estimate=False, ml_estimate_color="#4682b4",
            rasterized=False):
        samples = self.model.get_burned_in_samples()

        x_plot = x_axis_scale*np.asarray(x)

        if y is not None:
            plot(x_plot, y, color="r", lw=2, alpha=0.8, rasterized=rasterized)

        # Plot some samples onto the data.
        for idx, theta in enumerate(samples[np.random.randint(len(samples), size=nsamples)]):
        #for idx, theta in enumerate(samples):
            #print theta
            predictive_mu, sigma = self.model.predict_given_theta(x, theta)

            if idx == 0:
                plot(x_plot, predictive_mu, color=color, alpha=alpha, label=label, rasterized=rasterized)
            else:
                plot(x_plot, predictive_mu, color=color, alpha=alpha, rasterized=rasterized)
            fill_between(x_plot, predictive_mu-sigma, predictive_mu+sigma, color=color, alpha=0.01, rasterized=rasterized)

        if plot_ml_estimate:
            ml_theta = self.model.ml_curve_model.ml_params
            predictive_mu, sigma = self.model.predict_given_theta(x, ml_theta)
            plot(x_plot, predictive_mu, alpha=1.0, color=ml_estimate_color, lw=3, rasterized=rasterized)
            fill_between(x_plot, predictive_mu-sigma, predictive_mu+sigma, color=ml_estimate_color, alpha=0.3, rasterized=rasterized)

        
        if vline is not None:
            axvline(x_axis_scale*vline, color="k")
        ylim(0, 1)
        xlabel(xaxislabel)
        ylabel(yaxislabel)   
        tight_layout()



    def predictive_density_plot(self, x):
        x_lin =  linspace(0., 1., 100)
        plot(x_lin, self.model.predictive_density(x, x_lin));



class MCMCCurveModelCombinationPlotter(object):
    def __init__(self, model):
        self.model = model
        self.colors = ['r', 'g', 'b', 'y', 'cyan', 'magenta', 'Chocolate', 'Crimson', 'DeepSkyBlue', 'Khaki']
        
    def trace_plot(self, figsize=None):
        num_plots = self.model.ndim
        
        if figsize is None:
            figsize = (8, num_plots)
        fig, axes = subplots(num_plots, 1, sharex=True, figsize=figsize)

        labels = ["$%s$" % (param_name) for model in self.model.fit_models for param_name in model.function_params]
        titles = ["%s" % (model.name) for model in self.model.fit_models for param_name in model.function_params]
        labels.append("$sigma$")
        titles.append("")
        labels.extend(["$w%d$" % idx for idx in range(len(self.model.fit_models))])
        titles.extend(["%s weight" % model.name for model in self.model.fit_models])

        for idx, label, title in zip(range(self.model.ndim), labels, titles):
            axes[idx].set_title(title)
            axes[idx].set_ylabel(label)
            axes[idx].plot(self.model.mcmc_chain[:, :, idx].T, color="k", alpha=0.4)
            axes[idx].yaxis.set_major_locator(MaxNLocator(5))
            #axes[0].axhline(m_true, color="#888888", lw=2)
            
            if idx == num_plots-1:
                axes[idx].set_xlabel("step number")
        
        tight_layout(h_pad=0.0)
        

    def triangle_plot(self, labels=None):
        samples = self.model.get_burned_in_samples()
        if labels is None:
            labels = ["$%s$" % (param_name) for model in self.model.fit_models for param_name in model.function_params]
            labels.append("$sigma$")
            labels.extend(["$w%d$" % idx for idx in range(len(self.model.fit_models))])
            labels = greek_label_mapping(labels)
        fig = triangle.corner(samples,
            labels=labels)

    def weights_triangle_plot(self, labels=None, thin=1):
        samples = self.model.get_burned_in_samples()
        if labels is None:
            labels = ["w%d" % idx for idx in range(len(self.model.fit_models))]
            #labels = greek_label_mapping(labels)
            print(labels)
        fig = triangle.corner(
            samples[::thin,-len(self.model.fit_models):])#,
            #labels=labels)

    def weights_triangle_plot(self, labels=None, thin=1):
        samples = self.model.get_burned_in_samples()
        if labels is None:
            labels = ["w%d" % idx for idx in range(len(self.model.fit_models))]
            #labels = greek_label_mapping(labels)
            print(labels)
        fig = triangle.corner(
            samples[::thin,-len(self.model.fit_models):])#,
            #labels=labels)

    def posterior_plot(self, *args, **kwargs):
        self.posterior_sample_plot(*args, **kwargs)

    def posterior_sample_plot(self, x, y=None, vline=None, alpha=0.1, label="",
            xaxislabel="$x$", yaxislabel="$y$", color="k", x_axis_scale=0.1,
            x_lim=None, plot_individual=False, y_plot_lw=2,
            rasterized=False):
        samples = self.model.get_burned_in_samples()

        if x_lim is None:
            x_predict = x
        else:
            x_predict = np.arange(1, x_lim, 1)

        x = x_axis_scale*x
        x_plot = x_axis_scale*np.asarray(x_predict)

        # Plot some samples onto the data.
        for idx, theta in enumerate(samples[np.random.randint(len(samples), size=100)]):
            predictive_mu, sigma = self.model.predict_given_theta(x_predict, theta)

            if idx == 0:
                plot(x_plot, predictive_mu, color=color, alpha=alpha, label=label, rasterized=rasterized)
            else:
                plot(x_plot, predictive_mu, color=color, alpha=alpha, rasterized=rasterized)

            fill_between(x_plot, predictive_mu-2*sigma, predictive_mu+2*sigma, color=color,
                rasterized=rasterized, alpha=0.01)
            if not plot_individual:
                continue
            #plot the contributions of the individual models:
            model_params, sigma, model_weights = self.model.split_theta(theta)
            if self.model.normalize_weights:
                model_weight_sum = np.sum(model_weights)
                model_probs = [weight/model_weight_sum for weight in model_weights]
            else:
                model_probs = model_weights
            for fit_model, model_color, model_prob, params in zip(self.model.fit_models, self.colors, model_probs, model_params):
                #if fit_model.function.__name__ != "ilog2":
                #    continue
                try:
                    sub_model_predictive_mu = fit_model.function(x, *params)
                    #plot(x_plot, model_prob * sub_model_predictive_mu, color=model_color, alpha=alpha)
                    plot(x_plot, sub_model_predictive_mu, color=model_color, alpha=alpha)
                except:
                    print("error with model: ", fit_model.function.__name__)

        if y is not None:
            plot(x, y, color="r", lw=y_plot_lw, alpha=0.8, label="data")
        if vline is not None:
            axvline(x_axis_scale*vline, color="k")
        ylim(0, 1)
        xlabel(xaxislabel)
        ylabel(yaxislabel)   
        tight_layout()

    def ml_single_models_plot(self, x, y, vline=None, x_axis_scale=0.1):
        lin_comb = None
        x_plot = x_axis_scale*np.asarray(x)
        for m in self.model.fit_models:
            plot(x_plot, m.predict(x), alpha=0.7, label=m.function.__name__, lw=2)
            model_weight = 1. / len(self.model.fit_models)
            if lin_comb is None:
                lin_comb = model_weight * m.predict(x)
            else:
                lin_comb += model_weight * m.predict(x)
        plot(x_plot, m.predict(x), alpha=0.7, label="linear combination", lw=2)

        plot(x_plot, y, color="r", lw=2, alpha=0.8)
        if vline is not None:
            axvline(x_axis_scale*vline, color="k")
        ylim(0, 1)
        legend(loc=4)
        xlabel("$x$")
        ylabel("$y$")   
        tight_layout()


    def predictive_density_plot(self, x):
        x_lin =  linspace(0., 1., 100)
        plot(x_lin, self.model.predictive_density(x, x_lin));


class MCMCCurveMixtureModelPlotter(object):
    def __init__(self, model):
        self.model = model
        self.colors = ['r', 'g', 'b', 'y', 'cyan', 'magenta', 'Chocolate', 'Crimson', 'DeepSkyBlue', 'Khaki']
        
    def trace_plot(self, figsize=None):
        num_plots = self.model.ndim
        
        if figsize is None:
            figsize = (8, num_plots)
        fig, axes = subplots(num_plots, 1, sharex=True, figsize=figsize)

        labels = ["$%s$" % (param_name) for model in self.model.fit_models for param_name in model.all_param_names]
        titles = ["%s" % (model.name) for model in self.model.fit_models for param_name in model.all_param_names]
        labels.extend(["$w%d$" % idx for idx in range(len(self.model.fit_models))])
        titles.extend(["%s weight" % model.name for model in self.model.fit_models])

        for idx, label, title in zip(range(self.model.ndim), labels, titles):
            axes[idx].set_title(title)
            axes[idx].set_ylabel(label)
            axes[idx].plot(self.model.mcmc_chain[:, :, idx].T, color="k", alpha=0.4)
            axes[idx].yaxis.set_major_locator(MaxNLocator(5))
            #axes[0].axhline(m_true, color="#888888", lw=2)
            
            if idx == num_plots-1:
                axes[idx].set_xlabel("step number")
        
        tight_layout(h_pad=0.0)
        

    def triangle_plot(self):
        samples = self.model.get_burned_in_samples()
        
        labels = ["$%s$" % param_name  for model in self.model.fit_models for param_name in model.all_param_names]
        labels.extend(["$%s$" for model in self.model.fit_models])
        fig = triangle.corner(samples, labels=labels)


    def posterior_plot(self, *args, **kwargs):
        self.posterior_sample_plot(*args, **kwargs)

    def posterior_sample_plot(self, x, y, vline=None, alpha=0.8, label="", color="k", x_axis_scale=0.1):
        samples = self.model.get_burned_in_samples()

        x_plot = x_axis_scale*np.asarray(x)

        # Plot some samples onto the data.
        for idx, theta in enumerate(samples[np.random.randint(len(samples), size=100)]):
            predictive_mu = self.model.predict_given_theta(x, theta)

            if idx == 0:
                plot(x_plot, predictive_mu, color=color, alpha=alpha, label=label)
            else:
                plot(x_plot, predictive_mu, color=color, alpha=alpha)

        plot(x_plot, y, color="r", lw=2, alpha=0.8)
        if vline is not None:
            axvline(x_axis_scale*vline, color="k")
        ylim(0, 1)
        xlabel("$x$")
        ylabel("$y$")   
        tight_layout()


class EnsemblePlotter(object):

    def __init__(self,
                 ensemble_curve_model,
                 colors=['r', 'g', 'b', 'y', 'cyan', 'magenta', 'Chocolate', 'Crimson', 'DeepSkyBlue', 'Khaki']):
        self.ensemble_curve_model = ensemble_curve_model
        self.colors = colors
        assert len(colors) >= len(ensemble_curve_model.all_models), "Not enough colors for plot all fit models. Supply more colors!"
        #TODO: always use the same color for the same model!
        self.model_colors = {model.name: model_color for model, model_color in zip(ensemble_curve_model.all_models, colors)}

    def posterior_plot(self, x, y, vline=None, num_curves=100, x_label="epochs", y_label="accuracy", x_axis_scale=0.1):
        """
            x, y: data the posterior will be plotted upon.
            num_curves: the number of curves to plot
            x_axis_scale: scale the values on the xaxis (only for plotting but not passed to the function)
        """

        x_plot = x_axis_scale*np.asarray(x)

        for i in range(num_curves):
            predictive_mu, sigma, color = self.get_random_curve(x)

            plot(x_plot, predictive_mu, color=color, alpha=0.1)

            fill_between(x_plot, predictive_mu-2*sigma, predictive_mu+2*sigma, color=color, alpha=0.01)


        plot(x_plot, y, color="k", lw=2, alpha=0.8)
        if vline is not None:
            axvline(x_axis_scale*vline, color="k")
        ylim(0, 1)
        xlabel(x_label)
        ylabel(y_label)
        tight_layout()

    def get_random_curve(self, x):
        raise NotImplementedError("get_random_curve needs to be overriden")


class CurveModelEnsemblePlotter(EnsemblePlotter):

    def __init__(self, *args, **kwargs):
        super(CurveModelEnsemblePlotter, self).__init__(*args, **kwargs)

    def get_random_curve(self, x):
        """
            Sample a single curve under the given ensemble model.
        """
        #sample model:
        model_idx = np.random.multinomial(1, self.ensemble_curve_model.model_probabilities).argmax()
        model = self.ensemble_curve_model.fit_models[model_idx]
        if model.name in self.model_colors:
            model_color = self.model_colors[model.name]
        else:
            print("No color defined for model %s" % model.name)
        #sample curve:
        model_samples = model.get_burned_in_samples()   
        theta_idx = np.random.randint(0, model_samples.shape[0], 1)[0]
        theta = model_samples[theta_idx, :]

        params, sigma = model.split_theta(theta)
        predictive_mu = model.function(x, **params)
        return predictive_mu, sigma, model_color


class CurveEnsemblePlotter(EnsemblePlotter):
    def __init__(self, *args, **kwargs):
        super(CurveEnsemblePlotter, self).__init__(*args, **kwargs)


    def get_random_curve(self, x):
        #sample model:
        model_idx = np.random.multinomial(1, self.ensemble_curve_model.model_probabilities).argmax()
        model = self.ensemble_curve_model.fit_models[model_idx]
        model_theta_probabilities = self.ensemble_curve_model.model_theta_probabilities[model_idx]

        if model.name in self.model_colors:
            model_color = self.model_colors[model.name]
        else:
            print("No color defined for model %s" % model.name)

        model_samples = model.get_burned_in_samples()

        theta_idx = np.random.multinomial(1, model_theta_probabilities).argmax()
        theta = model_samples[theta_idx, :]
        params, sigma = model.split_theta(theta)
        predictive_mu = model.function(x, **params)
        return predictive_mu, sigma, model_color

**Model Factory**

In [9]:
def setup_model_combination(xlim,
        models=curve_combination_models,
		recency_weighting=False,
		normalize_weights=True,
		monotonicity_constraint=False,
		soft_monotonicity_constraint=True,
		nthreads=1):

    curve_models = []
    for model_name in models:
        if model_name == "linear":
            m = LinearCurveModel()
        else:
            if model_name in model_defaults:
                m = MLCurveModel(function=all_models[model_name],
                  default_vals=model_defaults[model_name],
                  recency_weighting=recency_weighting)
            else:
                m = MLCurveModel(function=all_models[model_name],
                  recency_weighting=recency_weighting)
        curve_models.append(m)

    model_combination = MCMCCurveModelCombination(curve_models,
        xlim=xlim,
        recency_weighting=recency_weighting,
        normalize_weights=normalize_weights,
        monotonicity_constraint=monotonicity_constraint,
        soft_monotonicity_constraint=soft_monotonicity_constraint,
        nthreads=nthreads)
    return model_combination


pool = None
def setup_curve_model(ensemble_type="curve_model", recency_weighting=False, pool_size=16):
    """
        type: either curve_model or curve
    """
    if pool_size > 1:
        pool = Pool(pool_size)
        map_fun = pool.map
    else:
        map_fun = map

    ensemble_models = []
    for model_name in curve_ensemble_models:
        if model_name == "linear":
            m = LinearCurveModel()
        else:
            if model_name in model_defaults:
                m = MLCurveModel(function=all_models[model_name],
                  default_vals=model_defaults[model_name],
                  recency_weighting=recency_weighting)
            else:
                m = MLCurveModel(function=all_models[model_name],
                  recency_weighting=recency_weighting)
        ensemble_models.append(m)

    if ensemble_type == "curve_model":
        ensemble_curve_model = CurveModelEnsemble(ensemble_models, map=map_fun)
    elif ensemble_type == "curve":
        ensemble_curve_model = CurveEnsemble(ensemble_models, map=map_fun)
    else:
        assert False, "unkown ensemble type"
    
    return ensemble_curve_model


def create_model(model_type, xlim, nthreads=1, recency_weighting=False):
	"""
		type: either curve_combination, curve_model or curve
				curve_combination: Bayesian Model curve_combination
				curve_model: Bayesian Model Averaging
		xlim: the target point that we want to predict eventually
		nthreads: 1 for no parallelization
	"""
	if model_type == "curve_combination":
		curve_model = setup_model_combination(
            xlim=xlim,
            recency_weighting=recency_weighting,
            nthreads=nthreads)
	elif model_type == "curve_model" or model_type == "curve":
		curve_model = setup_curve_model(
            ensemble_type=model_type,
            recency_weighting=recency_weighting,
            pool_size=nthreads)
	return curve_model


def create_plotter(model_type, model):
	if model_type == "curve_combination":
		return MCMCCurveModelCombinationPlotter(model)
	elif model_type == "curve_model" or model_type == "curve":
		return CurveModelEnsemblePlotter(ensemble_curve_model)

**Termination Criterion**

In [10]:
IMPROVEMENT_PROB_THRESHOLD = 0.05
PREDICTIVE_STD_THRESHOLD = 0.005

PREDICTION_THINNING = 10
NTHREADS = 4


def cut_beginning(y, threshold=0.05, look_ahead=5): #Identical to author's code

  # Number of steps so far should be greater than look_ahead
  if len(y) < look_ahead:
    return y

  num_cut = 0

  for idx in range(len(y)-look_ahead):
    start_here = True
    for idx_ahead in range(idx, idx+look_ahead):
      if not (y[idx_ahead] - y[0] > threshold):
        start_here = False
    if start_here:
      num_cut = idx
      break
  return y[num_cut:]

#Reimplemented:
def get_xlim(max_iter, test_interval):
  # Computes the number of iterative steps at which we want to predict the accuracy
  return max_iter / test_interval

# Mostly changed
class TerminationCriterion(object):
  def __init__(self, nthreads, prob_x_greater_type, max_iter, test_interval, predictive_std_threshold=PREDICTIVE_STD_THRESHOLD):
    self.prob_x_greater_type = prob_x_greater_type
    print("prob_x_greater_type: %s" % prob_x_greater_type)
    models = ["vap", "ilog2", "weibull", "pow3", "pow4", "loglog_linear",
              "mmf", "janoschek", "dr_hill_zero_background", "log_power",
              "exp4"]
    xlim = get_xlim(max_iter, test_interval)
    print("xlim: %d" % (xlim))
    self.xlim = xlim
    model = setup_model_combination(models=models,
                                    xlim=xlim,
                                    recency_weighting=True,
                                    nthreads=nthreads)
    self.model = model
    self.predictive_std_threshold = predictive_std_threshold

  def run(self, ybest, y):
    y_crt_best = np.max(y)
    y = cut_beginning(y)
    x = np.asarray(range(1, len(y)+1))

    if not self.model.fit(x, y):
      #failed fitting... not cancelling
      return 0

    predictive_std = self.model.predictive_std(self.xlim, thin=PREDICTION_THINNING)
    print("predictive_std: %f" % predictive_std)

    if predictive_std < self.predictive_std_threshold:
      #the model is pretty sure about the prediction: stop!
      print("predictive_std low, predicting and stopping...")
      return self.predict()
    elif ybest is not None:
      print("predictive_std high, let's check the probably to get higher than the current ybest")
      #we're still checking if maybe all the probability is below ybest
      if self.prob_x_greater_type == "posterior_prob_x_greater_than":
        prob_gt_ybest_xlast = self.model.posterior_prob_x_greater_than(self.xlim,
          ybest, thin=PREDICTION_THINNING)
      else:
        prob_gt_ybest_xlast = self.model.posterior_mean_prob_x_greater_than(self.xlim,
          ybest, thin=PREDICTION_THINNING)

      print("p(y>y_best) = %f" % prob_gt_ybest_xlast)

      if prob_gt_ybest_xlast < IMPROVEMENT_PROB_THRESHOLD:
        return self.predict()
      else:
        print("continue evaluating")
        #we are probably still going to improve
        return 0
    else:
      print("neither the predictive_std is low nor is there a ybest ... continue")
      return 0

  def predict(self):
    y_predict = self.model.predict(self.xlim, thin=PREDICTION_THINNING)
    print("probably only going to reach %f, stopping..." % y_predict)
    return y_predict

**Main**

In [ ]:
import random
def logpdf(y, loc=0, scale=1):
  y = (y - loc) / scale
  return math.exp(-y**2/2)/(math.sqrt(2)*math.pi)

def cdf(y, loc=0, scale=1):
  y = (y - loc) / scale
  l = 1 / np.mean(y)
  return 1 - math.exp(-l * y)

datasets = [3, 12, 23, 31, 54, 181, 188, 1457, 1461, 1464, 1468, 1475, 1485, 1486,
            1487, 1489, 1494, 1515, 1590, 4134, 4135, 4534, 4538, 4541, 23512,
            23517, 40498, 40668, 40670, 40685, 40701, 40900, 40975, 40978, 40981,
            40982, 40983, 40984, 40996, 41027, 41138, 41142, 41143, 41144, 41145,
            41146, 41147, 41156, 41157, 41158, 41159, 41161, 41162, 41163, 41164,
            41165, 41166, 41168, 41169, 42732, 42733, 42734]


ds = openml.datasets.get_dataset(12)
df = ds.get_data()[0].dropna()
y = np.array(df[ds.default_target_attribute].values)
X = np.array(df.drop(columns=[ds.default_target_attribute], axis=1))

learners = [LinearSVC(), DecisionTreeClassifier(), ExtraTreeClassifier(), ExtraTreesClassifier(), RandomForestClassifier(),
            GradientBoostingClassifier(), LogisticRegression(), PassiveAggressiveClassifier(), Perceptron(), RidgeClassifier(),
            SGDClassifier(), MLPClassifier(), LinearDiscriminantAnalysis(), QuadraticDiscriminantAnalysis(), BernoulliNB(),
            MultinomialNB(), KNeighborsClassifier()]

def scrambled(orig):
    dest = orig[:]
    random.seed(1)
    random.shuffle(dest)
    return dest

def results(dataset_id, learners=learners, folds=10):
  results = {}

  ds = openml.datasets.get_dataset(dataset_id)
  print(ds)
  df = ds.get_data()[0].dropna()
  y = np.array(df[ds.default_target_attribute].values)
  X = np.array(df.drop(columns=[ds.default_target_attribute], axis=1))
  X = scrambled(X)
  y = scrambled(y)
  X_train, y_train, X_test, y_test = train_test_split(X, y, 0.8)

  # See how long it takes to perform HPO without PTC
  random.seed(dataset_id)
  best_acc = 0
  l_best = None
  start_time = time.time()
  for i in range(30):
    print("ITERATION", i)
    l = clone(random.choice(learners))
    print("CHOSEN LEARNER:", l)
    accs = []
    l.fit(X_train, y_train)
    y_pred = l.predict(X_test)
    acc = accuracy_score(y_pred, y_test)
    if acc > best_acc:
      best_acc = acc
      print("NEW BEST ACCURACY: ", best_acc, "FOR MODEL", l)
      l_best = l
  train_time = time.time() - start_time
    
  results["no PTC learner"] = l_best
  results["no PTC accuracy"] = best_acc
  results["no PTC time"] = train_time

  # See how long it takes to perform HPO with PTC
  random.seed(dataset_id)
  best_acc = 0
  X_train = list(split(X_train, 10))
  y_train = list(split(y_train, 10))
  test_interval = 1
  folds = 10
  prob_x_greater_type = "posterior_prob_x_greater_than"
  nthreads = NTHREADS
  predictive_std_threshold = PREDICTIVE_STD_THRESHOLD
  l_best = None

  start_time = time.time()
  for i in range(30):
    print("ITERATION", i)
    l = clone(random.choice(learners))
    print("CHOSEN LEARNER:", l)
    term_crit = TerminationCriterion(nthreads,
                  prob_x_greater_type, folds, test_interval,
                  predictive_std_threshold=predictive_std_threshold)
    accs = []
    for f in range(folds):
      l.fit(X_train[f], y_train[f])
      y_pred = l.predict(X_test)
      accs.append(accuracy_score(y_pred, y_test))
      acc_pred = term_crit.run(best_acc, accs)
      if acc_pred < 0 or acc_pred > 1:
          continue
      print("Y POINTS:", accs)
      if not(acc_pred == 0.0):
        break
    if acc_pred==0.0:
      acc_pred = accs[-1]
    print("ACCURACY PREDICTED", acc_pred, "FOR MODEL", l)
    if acc_pred > best_acc:
      best_acc = acc_pred
      print("NEW BEST ACCURACY: ", best_acc, "FOR MODEL", l)
      l_best = l
  train_time = time.time() - start_time
    
  results["PTC learner"] = l_best
  results["PTC accuracy"] = best_acc
  results["PTC time"] = train_time
  return results



for dataset_id in [12,54,1464,1468,23517,40670,40900]:
    dic = results(dataset_id, learners, 10)
    print(dic)
    f = open('_results_'+str(dataset_id)+'.txt', 'w')
    f.write(str(dic))
    f.close() 